<a href="https://colab.research.google.com/github/Lonely52Hz/ID2223_Scalable_Machine_Learning_and_Deep_Learning/blob/main/ID2223_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# set environment in colab
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"
!pip install pyspark

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 54.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=d532112c70b76be186d38ddb174e3061578bcf014062751b8f50169a758d77a1
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Colab").config("spark.ui.port", "4050").getOrCreate()
from pyspark.sql.functions import *
#from pyspark.sql.types import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [44]:
# read files
train = spark.read.csv('drive/MyDrive/ID2223_File/stsbenchmark/sts-train.csv', sep = '\t', inferSchema = True)\
     .withColumnRenamed('_c0', 'genre').withColumnRenamed('_c1', 'file').withColumnRenamed('_c2', 'year')\
     .withColumnRenamed('_c3', 'index').withColumnRenamed('_c4', 'score')\
     .withColumnRenamed('_c5', 'sentenceA').withColumnRenamed('_c6', 'sentenceB')
test = spark.read.csv('drive/MyDrive/ID2223_File/stsbenchmark/sts-test.csv', sep = '\t', inferSchema = True)\
     .withColumnRenamed('_c0', 'genre').withColumnRenamed('_c1', 'file').withColumnRenamed('_c2', 'year')\
     .withColumnRenamed('_c3', 'index').withColumnRenamed('_c4', 'score')\
     .withColumnRenamed('_c5', 'sentenceA').withColumnRenamed('_c6', 'sentenceB')
dev = spark.read.csv('drive/MyDrive/ID2223_File/stsbenchmark/sts-dev.csv', sep = '\t', inferSchema = True)\
     .withColumnRenamed('_c0', 'genre').withColumnRenamed('_c1', 'file').withColumnRenamed('_c2', 'year')\
     .withColumnRenamed('_c3', 'index').withColumnRenamed('_c4', 'score')\
     .withColumnRenamed('_c5', 'sentenceA').withColumnRenamed('_c6', 'sentenceB')

train.show(5)
train.printSchema()

+-------------+------+--------+-----+-----+--------------------+--------------------+
|        genre|  file|    year|index|score|           sentenceA|           sentenceB|
+-------------+------+--------+-----+-----+--------------------+--------------------+
|main-captions|MSRvid|2012test|    1|  5.0|A plane is taking...|An air plane is t...|
|main-captions|MSRvid|2012test|    4|  3.8|A man is playing ...|A man is playing ...|
|main-captions|MSRvid|2012test|    5|  3.8|A man is spreadin...|A man is spreadin...|
|main-captions|MSRvid|2012test|    6|  2.6|Three men are pla...|Two men are playi...|
|main-captions|MSRvid|2012test|    9| 4.25|A man is playing ...|A man seated is p...|
+-------------+------+--------+-----+-----+--------------------+--------------------+
only showing top 5 rows

root
 |-- genre: string (nullable = true)
 |-- file: string (nullable = true)
 |-- year: string (nullable = true)
 |-- index: integer (nullable = true)
 |-- score: double (nullable = true)
 |-- sentenc

In [45]:
# map [0, 5] to [-1, 1]
train = train.withColumn('score', col('score')/2.5 - 1)
test = test.withColumn('score', col('score')/2.5 - 1)
dev = dev.withColumn('score', col('score')/2.5 - 1)

train.show(5)
train.describe(['score']).show()

+-------------+------+--------+-----+--------------------+--------------------+--------------------+
|        genre|  file|    year|index|               score|           sentenceA|           sentenceB|
+-------------+------+--------+-----+--------------------+--------------------+--------------------+
|main-captions|MSRvid|2012test|    1|                 1.0|A plane is taking...|An air plane is t...|
|main-captions|MSRvid|2012test|    4|                0.52|A man is playing ...|A man is playing ...|
|main-captions|MSRvid|2012test|    5|                0.52|A man is spreadin...|A man is spreadin...|
|main-captions|MSRvid|2012test|    6|0.040000000000000036|Three men are pla...|Two men are playi...|
|main-captions|MSRvid|2012test|    9|                 0.7|A man is playing ...|A man seated is p...|
+-------------+------+--------+-----+--------------------+--------------------+--------------------+
only showing top 5 rows

+-------+-------------------+
|summary|              score|
+-----

In [46]:
# find rows with missing value
train.select([count(when(col(c).isNull(), c)).alias(c) for c in train.columns]).show()
test.select([count(when(col(c).isNull(), c)).alias(c) for c in test.columns]).show()
dev.select([count(when(col(c).isNull(), c)).alias(c) for c in dev.columns]).show()

+-----+----+----+-----+-----+---------+---------+
|genre|file|year|index|score|sentenceA|sentenceB|
+-----+----+----+-----+-----+---------+---------+
|    0|   0|   0|    0|    0|        0|        6|
+-----+----+----+-----+-----+---------+---------+

+-----+----+----+-----+-----+---------+---------+
|genre|file|year|index|score|sentenceA|sentenceB|
+-----+----+----+-----+-----+---------+---------+
|    0|   0|   0|    0|    0|        0|        3|
+-----+----+----+-----+-----+---------+---------+

+-----+----+----+-----+-----+---------+---------+
|genre|file|year|index|score|sentenceA|sentenceB|
+-----+----+----+-----+-----+---------+---------+
|    0|   0|   0|    0|    0|        0|        3|
+-----+----+----+-----+-----+---------+---------+



In [51]:
# delete rows with missing value
train = train.na.drop()
test = test.na.drop()
dev = dev.na.drop()

train.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()
train.show(5)

+-----+----+----+-----+-----+---------+---------+
|genre|file|year|index|score|sentenceA|sentenceB|
+-----+----+----+-----+-----+---------+---------+
|    0|   0|   0|    0|    0|        0|        0|
+-----+----+----+-----+-----+---------+---------+

+-------------+------+--------+-----+--------------------+--------------------+--------------------+
|        genre|  file|    year|index|               score|           sentenceA|           sentenceB|
+-------------+------+--------+-----+--------------------+--------------------+--------------------+
|main-captions|MSRvid|2012test|    1|                 1.0|A plane is taking...|An air plane is t...|
|main-captions|MSRvid|2012test|    4|                0.52|A man is playing ...|A man is playing ...|
|main-captions|MSRvid|2012test|    5|                0.52|A man is spreadin...|A man is spreadin...|
|main-captions|MSRvid|2012test|    6|0.040000000000000036|Three men are pla...|Two men are playi...|
|main-captions|MSRvid|2012test|    9|     